In [12]:
import luigi
import pandas as pd
import numpy as np
import sys
sys.path.insert(0, '/home/fbuonerba/codes/')
from coinapi_v1 import CoinAPIv1
import datetime
from datetime import datetime, timedelta
import time
import calendar
import json
import urllib.request
import multiprocessing as mp

#REALtest_key = 'DB318A59-25FF-499E-9A6D-783A19C346D8'

test_key = 'EED0F746-36FB-4CC4-8E7C-527333DFA6FB'

api = CoinAPIv1(test_key)

meta_path='/home/fbuonerba/codes/meta_data/'
with open(meta_path+'top_coins.txt') as f:
    coins=json.load(f)
with open(meta_path+'symbols.txt') as ff:
    symbols=json.load(ff)
quotes=['USD','BTC']

In [14]:
len(symbols)

893

In [ ]:
#Luigi version.
#this task downloads data (by luigi default, if it does not exist already).
class request_exchange_rates(luigi.Task):
    unix_time=luigi.Parameter()
    base=luigi.Parameter()
    quote=luigi.Parameter()
    path='/home/fbuonerba/exchange_rates_data/'
    def requires(self):
        return None
    def output(self):
        return luigi.LocalTarget(self.path+'exchange_rate_'+str(self.base)+'_'+str(self.quote)+'_'+str(self.unix_time)+'.txt')
    def run(self):
        while True:
            try: 
                utctime = datetime.utcfromtimestamp(self.unix_time).strftime('%Y-%m-%dT%H:%M:%S')
                exchange=api.exchange_rates_get_specific_rate(self.base, self.quote, {'time': utctime})
                with self.output().open('w') as outfile:
                    json.dump(exchange, outfile) 
                print('okokokokokok', self.unix_time)
                break
            except urllib.error.HTTPError as err:
                print('AAAAAAAAAAAAAAAAAAAA', err, self.unix_time)
                time.sleep(30)
            
        
    

In [ ]:
unix_time= 1529985000
#1586918800  1529954000
while True:
    luigi.build([request_exchange_rates(unix_time + 600*t, base='ETH', quote='USD') for t in range(2)], workers=2, local_scheduler=True)
    unix_time+=1200
    #luigi.build([request_exchange_rates(unix_time + 600*t, base='ETH', quote='USD') for t in range(100)], workers=1, local_scheduler=True)
    

In [ ]:
######GET TOP 50 COINS#######

In [ ]:
usd_sym=[]
for sym in symbols:
    if sym['asset_id_quote']=='USD':
        usd_sym.append(sym['symbol_id'])

usd_sym

In [ ]:
########THIS REQUIRES PLENTY OF REQUESTS#########
#dicti={}
#for sym in usd_sym:
#    print(sym)
#    dicti[sym]=api.ohlcv_latest_data(sym, {'period_id': '1DAY', 'limit':1})

In [ ]:
assets = api.metadata_list_assets()
symbols = api.metadata_list_symbols()
exchanges = api.metadata_list_exchanges()

In [ ]:
with open('/home/fbuonerba/codes/some_ohlcv.txt') as ff:
    dicti=json.load(ff)

dicti['BITSTAMP_SPOT_BTC_USD']

In [ ]:
d={}
for asse in assets:
    asse_sum=0
    for sym in symbols:
        if sym['asset_id_quote']=='USD' and sym['asset_id_base']==asse['asset_id'] and sym['symbol_type']=='SPOT':
            s=sym['symbol_id']
            if (s in dicti) and dicti[s]!=[]:
                average=dicti[sym['symbol_id']][0]['price_high']+dicti[sym['symbol_id']][0]['price_low']
                asse_sum=asse_sum + dicti[sym['symbol_id']][0]['volume_traded']*average
    d[asse['asset_id']]=asse_sum


In [ ]:
items = [(v, k) for k, v in d.items()]
items.sort()
items.reverse()             # so largest is first
items = [(k, v) for v, k in items]
top_coins=[fin[0] for fin in items[0:50]]
top_coins

In [ ]:
with open('top_coins.txt','w') as file:
    json.dump(top_coins,file)


In [ ]:
top_exchanges=['BINANCE','BITFINEX','HUOBI','COINBASE','OKEX','BITSTAMP','HITBTC','BITHUMB','POLONIEX','KRAKEN']

with open('/home/fbuonerba/codes/top_exchanges.txt','w') as file:
    json.dump(top_exchanges,file)

In [ ]:
with open('/home/fbuonerba/codes/meta_data/top_coins.txt') as file:
    top_coins=json.load(file)

In [ ]:
top_coins+=['ADA','BNB']

In [ ]:
with open('/home/fbuonerba/codes/meta_data/top_coins.txt','w') as file:
    json.dump(top_coins,file)

In [ ]:
import os
count=0
S=os.listdir('/home/fbuonerba/log_returns_data')
for s in S[100:200]:
    with open('/home/fbuonerba/log_returns_data/' + s,'r') as file:
        data=file.read()
    if data=='{}':
        data=data.replace('{}','NaN')
    with open('/home/fbuonerba/log_returns_data/' + s,'w') as file:
        file.write(data)
    
        
    

In [4]:
unix_time=1522540800
utctime = datetime.utcfromtimestamp(unix_time).strftime('%Y-%m-%dT%H:%M:%S')
ohlcv=api.ohlcv_historical_data('BINANCE_SPOT_WAVES_BTC', {'period_id': '1DAY', 'time_start':utctime})

In [11]:
liss=[]
for coin in coins:
    sym='BINANCE_SPOT_'+str(coin)+'_BTC'
    try:
        X=api.ohlcv_historical_data(sym, {'period_id': '7DAY', 'time_start':utctime,'limit':15})
        print(coin, X)
        liss.append(X)
    except Exception as e:
        print(coin, e)

BTC HTTP Error 400: Bad Request
ETH [{'price_high': 0.057034, 'time_period_start': '2018-03-26T00:00:00.0000000Z', 'price_close': 0.055546, 'time_close': '2018-04-01T23:59:58.1160000Z', 'volume_traded': 105545.911, 'time_open': '2018-04-01T00:00:01.0950000Z', 'time_period_end': '2018-04-02T00:00:00.0000000Z', 'price_low': 0.0551, 'trades_count': 203935, 'price_open': 0.05686}, {'price_high': 0.057261, 'time_period_start': '2018-04-02T00:00:00.0000000Z', 'price_close': 0.05709, 'time_close': '2018-04-08T23:59:59.1590000Z', 'volume_traded': 828397.858, 'time_open': '2018-04-02T00:00:00.4350000Z', 'time_period_end': '2018-04-09T00:00:00.0000000Z', 'price_low': 0.053672, 'trades_count': 2147478, 'price_open': 0.055545}, {'price_high': 0.064686, 'time_period_start': '2018-04-09T00:00:00.0000000Z', 'price_close': 0.063889, 'time_close': '2018-04-15T23:59:59.3290000Z', 'volume_traded': 972927.867, 'time_open': '2018-04-09T00:00:00.6140000Z', 'time_period_end': '2018-04-16T00:00:00.0000000Z', 

LTC [{'price_high': 0.017125, 'time_period_start': '2018-03-26T00:00:00.0000000Z', 'price_close': 0.01686, 'time_close': '2018-04-01T23:59:55.0980000Z', 'volume_traded': 88434.95, 'time_open': '2018-04-01T00:00:15.4080000Z', 'time_period_end': '2018-04-02T00:00:00.0000000Z', 'price_low': 0.0166, 'trades_count': 26510, 'price_open': 0.016764}, {'price_high': 0.0183, 'time_period_start': '2018-04-02T00:00:00.0000000Z', 'price_close': 0.016751, 'time_close': '2018-04-08T23:59:59.1230000Z', 'volume_traded': 686526.02, 'time_open': '2018-04-02T00:00:01.6870000Z', 'time_period_end': '2018-04-09T00:00:00.0000000Z', 'price_low': 0.016608, 'trades_count': 218412, 'price_open': 0.016856}, {'price_high': 0.017169, 'time_period_start': '2018-04-09T00:00:00.0000000Z', 'price_close': 0.015851, 'time_close': '2018-04-15T23:59:58.2420000Z', 'volume_traded': 641428.7, 'time_open': '2018-04-09T00:00:01.0070000Z', 'time_period_end': '2018-04-16T00:00:00.0000000Z', 'price_low': 0.015651, 'trades_count': 2

EOS HTTP Error 429: Too Many Requests
ETC HTTP Error 429: Too Many Requests
XMR HTTP Error 429: Too Many Requests
NEO HTTP Error 429: Too Many Requests
BTG HTTP Error 429: Too Many Requests
USDT HTTP Error 429: Too Many Requests
ZEC HTTP Error 429: Too Many Requests
EUR HTTP Error 429: Too Many Requests
IOTA HTTP Error 429: Too Many Requests
DASH HTTP Error 429: Too Many Requests
DIG HTTP Error 429: Too Many Requests
XLM HTTP Error 429: Too Many Requests
HBZ HTTP Error 429: Too Many Requests
OMG HTTP Error 429: Too Many Requests
TRX HTTP Error 429: Too Many Requests
ODE HTTP Error 429: Too Many Requests
BFT HTTP Error 429: Too Many Requests
ELF HTTP Error 429: Too Many Requests
GNT HTTP Error 429: Too Many Requests
DOGE HTTP Error 429: Too Many Requests
SNT HTTP Error 429: Too Many Requests
DAT HTTP Error 429: Too Many Requests
REP HTTP Error 429: Too Many Requests
EMC HTTP Error 429: Too Many Requests
AIO HTTP Error 429: Too Many Requests
XVG HTTP Error 429: Too Many Requests
RLC HTTP

In [10]:
len(lis)

31